In [1]:
!pip install gtts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [7]:
import moviepy.editor as mp
import gtts
from PIL import Image, ImageDraw, ImageFont

# List of images and their respective durations in seconds
images = [
    ("./data/pet_morning.png", 6),
    ("./data/pet_nap.png", 12),
    ("./data/pet_playtime.png", 4)
]

# Corresponding voiceover text for each image
voiceover_texts = [
    "My day begins with a gentle wake-up call, usually from the sound of my food bowl being filled.",
    "After a busy morning, it's crucial to find the perfect spot for an afternoon nap. My favorite is a sunbeam near the window.",
    "The best part of the day is when my human comes home. It's finally time for our evening game of fetch!"
]

# Captions for each image
captions = [
    "My day begins with a gentle wake-up call, usually from the sound of my food bowl being filled.",
    "After a busy morning, it's crucial to find the perfect spot for an afternoon nap. My favorite is a sunbeam near the window.",
    "The best part of the day is when my human comes home. It's finally time for our evening game of fetch!"
]

# Function to add text to an image with multiple lines and black rectangle background
def add_text_to_image_multiline_black(image_path, text, output_path, font_size=40, max_width=50, y_offset=400):
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("LiberationSans-Bold.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()
        print("Warning: LiberationSans-Bold.ttf not found, using default font.")


    # Split text into multiple lines
    lines = []
    words = text.split()
    while words:
        line = ''
        while words and len(line) + len(words[0]) + 1 <= max_width:
            line = line + (words.pop(0) + ' ')
        lines.append(line.strip())

    # Calculate text position
    total_text_height = sum([draw.textbbox((0, 0), line, font=font)[3] - draw.textbbox((0, 0), line, font=font)[1] for line in lines])
    width, height = img.size
    y = (height - total_text_height) / 2 + y_offset

    for line in lines:
        text_w = draw.textlength(line, font=font)
        text_h = draw.textbbox((0, 0), line, font=font)[3] - draw.textbbox((0, 0), line, font=font)[1]
        x = (width - text_w) / 2
        draw.rectangle([(x - 10, y - 10), (x + text_w + 10, y + text_h + 10)], fill="black")
        draw.text((x, y), line, font=font, fill="white")
        y += text_h * 1.2 # Reduced line spacing

    img.save(output_path)

# Add captions to images
captioned_images = []
for i, (image_path, duration) in enumerate(images):
    output_path = f"./data/captioned_pet_image_{i}.png"
    add_text_to_image_multiline_black(image_path, captions[i], output_path)
    captioned_images.append((output_path, duration))

# Generate voiceover using gTTS
voiceover_clips = []
for i, text in enumerate(voiceover_texts):
    tts = gtts.gTTS(text, lang='en')
    tts.save(f"./data/pet_voiceover_{i}.mp3")
    voiceover_clips.append(mp.AudioFileClip(f"./data/pet_voiceover_{i}.mp3"))

# Create video clips for each image with captions
image_clips = []
for i, (image_path, duration) in enumerate(captioned_images):
    img_clip = mp.ImageClip(image_path).set_duration(duration)
    image_clips.append(img_clip)

# Concatenate video clips
final_video_clip = mp.concatenate_videoclips(image_clips)

# Concatenate audio clips
final_audio_clip = mp.concatenate_audioclips(voiceover_clips)

# Set the audio of the final video clip
final_clip = final_video_clip.set_audio(final_audio_clip)

# Write the final video to a file
output_file_path = "./day_in_life_pet.mp4"
final_clip.write_videofile(output_file_path, fps=24)

output_file_path

Moviepy - Building video ./day_in_life_pet.mp4.
MoviePy - Writing audio in day_in_life_petTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video ./day_in_life_pet.mp4



Moviepy - Done !
Moviepy - video ready ./day_in_life_pet.mp4


'./day_in_life_pet.mp4'